In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=4e418ed0b362f57c300a7968cecb8127162bf1df308a9a15f66b6eabb78d6739
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
# import all libraries
from google.colab import files
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice
import math
import pandas as pd
import numpy as np
import yfinance as yf

In [8]:
uploaded = files.upload()

Saving df0.csv to df0.csv


In [9]:
df = pd.read_csv('df0.csv')

In [10]:
df.head()

,index,list
0,0,"['AVGO', 'HUM', 'APH', 'APTV', 'MSCI', 'GD', '..."
1,1,"['BAC', 'WBD', 'DXCM', 'EMR', 'CVS', 'PCAR', '..."
2,2,"['NWSA', 'GWW', 'FAST', 'REGN', 'MAS', 'EQT', ..."
3,3,"['HOLX', 'NDSN', 'HES', 'TROW', 'TRGP', 'KMX',..."
4,4,"['IT', 'MS', 'SCHW', 'GILD', 'UNH', 'APD', 'MD..."


In [12]:
df.iloc[0].list

"['AVGO', 'HUM', 'APH', 'APTV', 'MSCI', 'GD', 'POOL', 'CRL', 'ITW', 'PEP']"

In [13]:
class Common_Class():
  """
  A common class to calculate the results for all strategies.
  Key Terms;
      - A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
  Methods in class:
  1. prepare_data()
      - Loads the close prices of all stocks and calculates the return of each stock
  2. close_graph(stock)
      - A line plot showcasing the close prices of all stocks over the stated period (start - end)
  3. return_date_price(stock)
      - Returns the date and price of the stock at that given date
  4. realised_balance(bar)
      - Returns the realised capital in your account in a given bar
  5. unrealised_balance(bar)
      - Returns the unrealised capital in your account in a given bar
  6. total_balance(bar)
      - Unrealised + Realised
  7. store_results()
          - Stores the results in a dataframe
  8. performance()
      - Calculates the performance ratios that is used to evaluate performance of strategy
      - Ratios used:
          1. Sharpe Ratio
          2, ROI (Return on investment)
          3. Drawdown
  9. performance_plots()
      - Visualisations of returns of stocks VS strategy
      - Plot 1: Movement of close prices of all stocks over stated period
      - Plot 2: Histogram of strategy returns
      - Plot 3: Strategy vs Instrument returns
      - Plot 4: Drawdown
      - Plot 5: Equity Curve
  10. buy_order()
      - Conducts a buy order upon fulfilling buy conditions of strategy
  11. sell_order()
      - Conducts a sell order upon fulfilling sell conditions of strategy
  12. last_trade()
      - Checks the last trade conducted
  """
  def __init__(self, symbol, start, end, interval, capital, transcation_cost, verbose = True):

    #new dicts as now there will be multiple stocks. Each stock will have its own position/quantity etc
    self.all_data ={}
    self.position = {}
    self.quantity = {}
    self.stoptrade = {}
    self.boughtprice = {}

    for stock in symbol:
      self.quantity['{}'.format(stock)] = 0 #can also do at run strategy, but its different design

    self.symbol = symbol
    self.start = start
    self.end = end
    self.interval = interval
    self.initial_capital = capital # this is the initial capital you want to trade with
    self.capital = capital # this capital will change depending on trades
    self.transaction_cost = transcation_cost # the transaction cost for trading
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)
    self.stored_data = pd.DataFrame(columns = ['trade', 'date', 'position', 'price', 'symbol', 'quantity', 'capital']) # to store trade details

    self.prepare_data() # prepares the data

  def prepare_data(self):
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    for i in self.symbol:
      stock_data = yf.Ticker(i)
      hist_stock = stock_data.history(start = self.start, end = self.end, interval = self.interval)
      bt_data = pd.DataFrame()
      bt_data["Close_Price"] = hist_stock["Close"]
      bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
      bt_data = bt_data.dropna()
      self.all_data['{}'.format(i)] = bt_data

  def close_graph(self, stock):
    i = str(stock)
    plt.figure(figsize=(15, 5))
    plt.plot(self.all_data[i]["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(i))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar, stock): #now we will also pass the stock as parameter
    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date
    i = str(stock)
    date = str(self.all_data[i].index[bar])[:10]  # :10 to only get the date
    price = self.all_data[i].Close_Price.iloc[bar]
    return date, price

  def realised_balance(self, bar):
    #returns the realised capital in your account at a given time period / bar
    date, price = self.return_date_price(bar, self.symbol[0]) # only concerned with date so any stock will do
    print("Date :{} | Realised Balance: {:0.1f}".format(date,self.capital))

  def unrealised_balance(self, bar):
    #returns you the unrealised capital (trades in progress) in your account at a given time period / bar
    ub = 0
    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      ub =  ub + self.quantity['{}'.format(stock)] *price
    #print("Date :{} | Unrealised Balance: {:0.1f}".format(date,ub))

  def total_balance(self, bar):
    #Unrealised plus realised
    ub = 0
    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      ub =  ub + self.quantity['{}'.format(stock)] *price
    tb = ub + self.capital
    #print("Date :{} | Total Balance: {:0.1f}".format(date,tb))
    return tb

  def store_results(self, trade, date, position, price, symbol, quantity, capital):
    trade_detail = pd.DataFrame({'trade':pd.Series(trade), 'date':pd.Series(date), 'position':pd.Series(position), 'price':pd.Series(price), 'symbol':pd.Series(symbol), 'quantity':pd.Series(quantity), 'capital':pd.Series(capital) })
    self.stored_data = pd.concat([self.stored_data, trade_detail])

  def performance_plots(self):# once we have dataframe of daily account balances
    Final_result_data = self.Final_result_data #saving time to write self everywhere
    interval = str(self.interval)
    f,ax = plt.subplots(2,3,figsize=(20,10))

    #plot 1: Close Price os stock
    plt.subplot(2, 3, 1)
    for stock in self.symbol:
      plt.plot(Final_result_data['Returns_{}'.format(stock)].cumsum(), label='{}'.format(stock), linestyle='dashed')
    plt.title("Cum Log returns of included stocks")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 2: Histogram of dtrstegy returns
    plt.subplot(2, 3, 2)
    plt.hist(Final_result_data['capital_log_returns'], bins = 35, label='Strategy Returns', linestyle='dashed')
    plt.title("Histogram of Strategy returns")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 3: Strategy vs Instrument
    plt.subplot(2, 3, 3)
    plt.plot(Final_result_data['Total_Stock_Returns_cum_sum'],color='red', label='Equal weighted stock Returns', linestyle='dashed')
    plt.plot(Final_result_data['capital_regular_returns_cumsum'].dropna(), label ='Strategy Returns')
    plt.title("Strategy Vs Stock Cum Return Curve")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 4: Drawdown
    plt.subplot(2, 3, 4)
    plt.plot(Final_result_data["capital_returns_log_cum_sum"].dropna(), label='Cum log returns')
    plt.plot(Final_result_data["Cum_Max"].dropna(), label =  'Max Drawdown')
    plt.title("Drawdown")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    #plot 5: Equity Curve
    plt.subplot(2, 3,  5)
    plt.plot(Final_result_data["capital"].dropna(), label =  'Equity $')
    plt.title("Equity Curve")
    plt.xlabel(interval)
    plt.legend()
    plt.grid()

    plt.suptitle("Performance Plots")
    f.delaxes(ax[1,2]) # to delete the last subplot

    plt.show()

  def performance(self):
    # stored data has  [trade, date, position, price, symbol, quantity, capital]
    #B = self.stored_data # this will have many rows ; n(number of instruments) times more
    #C = self.all_data # will need this to aggregate the returns from all stocks
    #B.index = range(B.shape[0]/len(self.symbol))

    #Since the stocks are not stored in arranged manner, we need to arrange them so that we can get the entry price and exit price for each day
    #stored data has many rows, we will aggregate day wise

    df = pd.DataFrame() # creating a new datframe to store aggregated values

    df['capital'] = self.stored_data.groupby('date').agg({'capital': 'mean'})
    # agrregating by mean because the run is done in loop (for all stocks in a list of symbols),
    # so each stock "date" will have the same capital accounting for all stocks
    # you can print and see different metrics..

    # df.index = range(self.stored_data.shape[0]/len(self.symbol)) # shape[0] to get the length of rows of each stock and make an index for dataframe df

    trade_start_date = df['capital'].index[0] # If intraday, then index needs to be adjusted with time, for daily, its fine
    trade_end_date = df['capital'].index[-1] # This will work when interval is 1D

    #Also, since we have multiple stocks, we need to calculate expected returns to comapre against the strategy

    for stock in self.symbol:
      A = (self.all_data[stock][trade_start_date:trade_end_date]['Return'].tolist()) # The returns are already log
      df['Returns_{}'.format(stock)] = A
    c= df.columns #fetch all the columns of dataframe
    df['Total_Stock_Returns'] = (df.loc[:,c[-len(self.symbol):]].sum(1))/len(self.symbol) # Sums up last N columns of df and divide by number of stocks. Equal weightage
    df['Total_Stock_Returns_cum_sum'] = df['Total_Stock_Returns'].cumsum()
    df.index = range(df.shape[0])

    df['capital_after_settlement'] = df['capital'].shift(-1)

    # Since we are are utilising capital, we will calculate returns for SR based on daily capital/equity changes
    #Remember, you can change the returns on investment based on the capital you trade

    df['capital_regular_returns'] = (df['capital_after_settlement']/df['capital'] - 1)
    df['capital_regular_returns_cumsum'] = df['capital_regular_returns'].cumsum()
    df['capital_log_returns'] = np.log(df['capital_after_settlement']/df['capital'])
    df["capital_returns_log_cum_sum"] = df["capital_log_returns"].cumsum()

    # for drawdown calculation
    df["Cum_Max"] = df["capital_returns_log_cum_sum"].cummax()
    df["drawdown"] = df["Cum_Max"] - df["capital_returns_log_cum_sum"]
    # The maximum difference is refered as "Drawdown"

    df.drop(df.index[df['capital_log_returns'] == 0.0], inplace=True) #Dropping the last column, as I already got the values through shift(1)
    df = df.dropna()
    #print(df.head())

    '''
    #arranging the columns for better readeability
    cols =['trade','symbol','position','quantity','date', 'exit_date', 'price','exit_price',\
           'price_log_returns','price_log_returns_cum_sum','capital',\
          'capital_after_settlement', 'capital_regular_returns', 'capital_log_returns',\
           'capital_returns_log_cum_sum','Cum_Max','drawdown']
    B = B[cols]
    B.rename(columns = {'date':'entry_date', 'price':'entry_price'}, inplace = True)

    '''

    #storing the results in Final_result data
    Final_result_data = df
    #Final_result_data.head()

    #Plot the equity curve
    #plt.plot(Final_result_data['exit_date'],Final_result_data['capital_after_settlement'])
    #Final_result_data[['exit_date', 'capital_after_settlement']].dropna().plot(figsize =(15,6), title = "Equity /Capital Plot")

    print("=" *50)
    print("Performance Metrics")

    #Total Balance
    print("The total capital at end of strategy: {:0.1f}".format(self.capital))

    # Annual gross expected returns
    daily_log_mean_ret = Final_result_data['capital_log_returns'].mean()
    annual_log_ret = 252 * daily_log_mean_ret
    annual_regular_return = (np.exp(annual_log_ret) -1)*100
    print("The annual regular returns: {:0.2f} %".format(annual_regular_return))

    # Annual std deviation
    daily_regular_std = (np.exp(Final_result_data['capital_log_returns'])-1).std()
    annual_regular_std =  daily_regular_std * (252 **0.5) *100
    print("The annual regular std deviation: {:0.2f} %".format(annual_regular_std))

    #Annual Sharpe Ratio
    SR = annual_regular_return / annual_regular_std
    print("The annual Sharpe Ratio: {:0.2f}".format(SR))

    #The strategy returns on investment in percent

    #Total_Log_ret = Final_result_data['capital_log_returns'].sum()
    #Total_Log_ret
    #Regular_ROI = (np.exp(Total_Log_ret) - 1) *100
    #print("The returns on investment from {} to {} by this strategy is {:0.2f} %".format(self.start, self.end, Regular_ROI))

    # the above method will also produce same result

    returns = (self.capital - self.initial_capital) /self.initial_capital *100
    print("The returns on investment from {} to {} by this strategy is {:0.1f} %".format(self.start, self.end, returns))

    #Calculate Drawdown
    #Final_result_data[["capital_returns_log_cum_sum", "Cum_Max"]].dropna().plot(figsize =(15,6), title = "Drawdown Plot")

    Drawdown = Final_result_data["drawdown"].max()*100
    print("The maximum drawdown: {:.2f} %".format(Drawdown))

    zero_periods = Final_result_data[Final_result_data['drawdown'] == 0]
    delta_values = (zero_periods.index[1:] - zero_periods.index[:-1])
    print("The maximum drawdown period: {:.2f} days".format(delta_values.max()))

    #Annual Sortino Ratio
    #Calculate the sortino ratio from the Final result data
    #sortino = ...
    #print("The annual Sortino Ratio: {:0.2f}".format(sortino))

    #Calculate any other ratio you want...

    #Total trades
    print("The total trades taken by the strategy: {}".format(self.trades))


    print("=" *50)

    self.Final_result_data = Final_result_data
    #print(self.Final_result_data.head())


  def buy_order(self,bar,stock,quantity=None, dollar =None ):
    date, price = self.return_date_price(bar,stock)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital - ((quantity * price)*(1 + self.transaction_cost)) # capital will be lost in buying
    self.quantity['{}'.format(stock)] = self.quantity['{}'.format(stock)] + quantity
    self.trades = self.trades + 1
    self.position['{}'.format(stock)] = 1
    tb = self.total_balance(bar)
    #self.quantity['{}'.format(stock)]*price + self.capital # this is just for one stock, however, I need for all the stocks
    self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, quantity, tb)
    self.boughtprice['{}'.format(stock)] = price


    if self.verbose:
      print("Bought {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,stock, price, quantity * price))
      print('Total quantity for {} is {}'.format(stock,self.quantity['{}'.format(stock)]))
      self.realised_balance(bar)
      #self.unrealised_balance(bar)
      #self.total_balance(bar)


  def sell_order(self,bar,stock, quantity=None, dollar=None ):
    date, price = self.return_date_price(bar, stock)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital + ((quantity * price)*(1 - self.transaction_cost)) # capital will be added after selling
    self.quantity['{}'.format(stock)] = self.quantity['{}'.format(stock)] - quantity
    self.trades = self.trades + 1
    self.position['{}'.format(stock)] = -1
    tb = self.total_balance(bar) # to store the total balance due to all the stocks at the end of the day, so that I get 1 value at the EOD
    self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, quantity, tb)
    self.boughtprice['{}'.format(stock)] = price


    if self.verbose:
      print("Sold {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,stock, price, quantity * price))
      print('Total quantity for {} is {}'.format(stock,self.quantity['{}'.format(stock)]))
      self.realised_balance(bar)
      #self.unrealised_balance(bar)
      #self.total_balance(bar)

  def last_trade(self, bar):# thi sneeds to be check for all open positions of all stocks

    for stock in self.symbol:
      date, price = self.return_date_price(bar, stock)
      last_quantity = self.quantity['{}'.format(stock)]
      self.capital = self.capital + last_quantity * price
      self.quantity['{}'.format(stock)] = 0 # as no more quantity now. all will be settled
      self.trades = self.trades +1
      if self.position['{}'.format(stock)] == -1: # if closing out a short position
        last_quantity = - last_quantity # to keep a positve nymber in records. Note, self.quantity is always maintianing a +/- sign.
        self.position['{}'.format(stock)] = 1 # to record we are buying to close out
      else:
        last_quantity =  last_quantity # to keep a positve nymber in records
        self.position['{}'.format(stock)] = -1 # to record we are selling to close out
      #self.position = 0
      tb = self.total_balance(bar)
      self.store_results(self.trades, date, self.position['{}'.format(stock)], price, stock, last_quantity, tb)

      if self.verbose:
        print("Closed open trades for {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(last_quantity,stock, price, last_quantity * price))

In [14]:
class MA_RSI_Strategy(Common_Class):
  """
    A class dictating our strategy conditions and the corresponding actions. This class inherits the Common_Class above.
    Methods in class:
    1. go_long(bar, stock):
        - Actions to conduct when a long position is taken
    2. go_short(bar, stock):
        - Actions to conduct when a short position is taken
    3. run_strategy(STMA_window, LTMA_window, RSI_window, upper_bound, lower_bound, stop_loss, take_profit):
        - Strategy conditions:
            - Long: LTMA > STMA and RSI < lower_bound
            - Short: LTMA < STMA and RSI > upper_bound
  """

  def go_long(self, bar, stock, quantity = None, dollar = None): # either previously in position or fresh position
    if self.position['{}'.format(stock)] == -1:
      self.position['{}'.format(stock)] = 0
      self.buy_order(bar,stock, quantity = -self.quantity['{}'.format(stock)]) #to clear previous short position and therefore negative quantity.(already in position, thats why negative)
      if dollar == 0: # stop loss or take profit condition
        self.stoptrade['{}'.format(stock)] = -1
        return None
    if quantity:
      self.buy_order (bar, stock, quantity = quantity) # to create new fresh order
      self.stoptrade['{}'.format(stock)] = 0
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      else:
        dollar = self.capital/len(self.symbol)
      self.buy_order(bar, stock, dollar = dollar)
      self.stoptrade['{}'.format(stock)] = 0

  def go_short(self, bar, stock, quantity = None, dollar = None):
    if self.position['{}'.format(stock)] == 1:
      self.position['{}'.format(stock)] = 0
      self.sell_order(bar,stock, quantity = self.quantity['{}'.format(stock)]) #to clear previous long vposition
      if dollar == 0: # stop loss or take profit condition
        self.stoptrade['{}'.format(stock)] = 1
        return None
    if quantity:
      self.sell_order (bar, stock, quantity = quantity) # to create new fresh order
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      else:
        dollar = self.capital/len(self.symbol)
      self.sell_order(bar, stock, dollar = dollar)
      self.stoptrade['{}'.format(stock)] = 0

  def run_strategy(self, STMA_window, LTMA_window, RSI_window, upper_bound, lower_bound, stop_loss, take_profit): # add the parameters according to the strategy
    self.stop = False # I will use this to make sure NO trading when balance = 0 OR lost all money
    self.trades = 0
    self.capital = self.initial_capital


    #Preparing data with LTMA and STMA so that can check crossovers. Also initialising with 0 quantity and 0 position

    for stock in self.symbol:
      self.quantity['{}'.format(stock)] = 0
      self.position['{}'.format(stock)] = 0

      indicator_1 = SMAIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window = STMA_window, fillna= False)
      STMA = indicator_1.sma_indicator()

      indicator_2 = SMAIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window = LTMA_window, fillna= False)
      LTMA = indicator_2.sma_indicator()

      self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)] = STMA
      self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)] = LTMA

      C = RSIIndicator(close = self.all_data['{}'.format(stock)]["Close_Price"], window= RSI_window, fillna = False)
      self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)] = C.rsi()

      #initialise the bought-in price and indicator for take profit/stop loss
      self.stoptrade['{}'.format(stock)] = 0 #1 indicates a previous take profit/stop loss action at long position, -1 indicates a previous action at short positin
      date, price = self.return_date_price(max(LTMA_window, RSI_window), stock) #initialise with first price
      self.boughtprice['{}'.format(stock)] = price

    for bar in range(max(LTMA_window, RSI_window), len(self.all_data['{}'.format(self.symbol[0])])): # from the available data bar to the last data bar
      #Making a func so that no trade if balance goes below 0

      tb = self.total_balance(bar)
      if tb < 0:
        print("negative capital")
        self.last_trade(bar)
        self.stop = True
        break

      for stock in self.symbol:
        date, price = self.return_date_price(bar, stock)


        if self.position['{}'.format(stock)] in [0,-1]: # checking no position or short position
          if self.position['{}'.format(stock)] in [-1]: # check for price change to stop loss/take profit when in position
            # buy to stop loss or take profit
            if price/self.boughtprice['{}'.format(stock)] - 1 >= stop_loss or price/self.boughtprice['{}'.format(stock)] - 1 <= -take_profit:
              self.go_long(bar,stock, dollar = 0)
          # go long if condition is satisfied and the previous trade is not take profit/stop loss at long position
          if (self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)].iloc[bar] > self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)].iloc[bar]\
          or self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)].iloc[bar] < lower_bound) and self.stoptrade['{}'.format(stock)] != 1:
             self.go_long(bar,stock, dollar = self.capital/len(self.symbol)) # go with equal money # Make sure to enter less than initila capital
             #print("--------")
             continue # as I am storing data for SR calculations. Dont wnat duplication

        if self.position['{}'.format(stock)] in [0,1]: # checking no position or long position
          if self.position['{}'.format(stock)] in [1]: # check for price change to stop loss/take profit when in position
            # sell to stop loss or take profit
            if price/self.boughtprice['{}'.format(stock)] - 1 <= -stop_loss or price/self.boughtprice['{}'.format(stock)] - 1 >= take_profit:
              self.go_short(bar,stock, dollar = 0)
          if (self.all_data['{}'.format(stock)]['STMA_{}'.format(stock)].iloc[bar] < self.all_data['{}'.format(stock)]['LTMA_{}'.format(stock)].iloc[bar]\
            or self.all_data['{}'.format(stock)]['RSI_{}'.format(stock)].iloc[bar] > upper_bound) and self.stoptrade['{}'.format(stock)] != -1:
            self.go_short(bar,stock, dollar =self.capital/len(self.symbol)) # go with equal money
            #print("--------")
            continue

        #-----Storing all the values for calculating Sharpe-----
        store_quantity = abs(self.quantity['{}'.format(stock)]) # so that I want just the positive numbers in my performance table. I will calculate +/- based on position sign
        tb = self.total_balance(bar)
        self.store_results(0, date, self.position['{}'.format(stock)], price, stock, store_quantity, tb)

    #print("=" *50)
    #print("End of last bar") Now the bar ends, bar is at the last bar value.
    #print("=" *50)
    if self.stop == False: # to make sure not to run this func 2 times when early exit due to negative balance
      self.last_trade(bar)  # amke sure the crossover happens, give more time


In [68]:
top_20_models = []

first_10 = df.iloc[:10].list
second_10 = df.iloc[10:20].list
third_10 = df.iloc[20:30].list
fourth_10 = df.iloc[30:40].list
fifth_10 = df.iloc[40:50].list
sixth_10 = df.iloc[50:60].list
seventh_10 = df.iloc[60:70].list
eighth_10 = df.iloc[70:80].list
nineth_10 = df.iloc[80:90].list
tenth_10 = df.iloc[90:100].list
eleventh_10 = df.iloc[100:110].list
twelfth_10 = df.iloc[110:120].list
thirteenth_10 = df.iloc[120:130].list
fourtheeth_10 = df.iloc[130:140].list
fifteenth_10 = df.iloc[140:150].list
sixteenth_10 = df.iloc[150:160].list
seventeeth_10 = df.iloc[160:170].list
eighteenth_10 = df.iloc[170:180].list
nineteenth_10 = df.iloc[180:190].list
twentieth_10 = df.iloc[190:].list

## 1st Run

In [107]:
model_1 =[]

for i in range(len(first_10)):
    stocks = eval(first_10[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_1.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [108]:
model_result = np.array(model_1)
sorted_index = np.argsort(model_1)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'first_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 6 is 30.592934116935734
ROI for model 0 is 17.792504922103884
ROI for model 4 is 11.65541777153015
ROI for model 2 is 7.584734136581421
ROI for model 5 is 5.318076637268066
ROI for model 7 is 2.912660948562622
ROI for model 8 is 0.519948328781128
ROI for model 3 is -0.30551379470825196
ROI for model 9 is -0.445354133605957
ROI for model 1 is -13.408876718330385


## 2nd Run

In [110]:
model_2 =[]

for i in range(len(second_10)):
    stocks = eval(second_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_2.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [111]:
model_result = np.array(model_2)
sorted_index = np.argsort(model_2)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'second_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 9 is 39.66862585971355
ROI for model 3 is 18.419799314498903
ROI for model 0 is 18.16322188053131
ROI for model 1 is 16.116532579898834
ROI for model 5 is 8.029873980331422
ROI for model 4 is 5.563144365692139
ROI for model 2 is 2.7349693669319155
ROI for model 7 is 2.037829693222046
ROI for model 8 is -9.50021534614563
ROI for model 6 is -16.532671475124356


## 3rd Run

In [112]:
model_3 =[]

for i in range(len(third_10)):
    stocks = eval(third_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_3.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [113]:
model_result = np.array(model_3)
sorted_index = np.argsort(model_3)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'third_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 4 is 19.200533485031126
ROI for model 3 is 13.383700573921203
ROI for model 8 is 11.365467230987548
ROI for model 0 is 5.895150170135498
ROI for model 9 is 5.346775785255432
ROI for model 1 is 4.839298253631592
ROI for model 5 is 3.287730080986023
ROI for model 6 is -5.575477416801452
ROI for model 2 is -7.370389498901367
ROI for model 7 is -10.533755632400513


## Run 4

In [69]:
model_4 =[]

for i in range(len(fourth_10)):
    stocks = eval(fourth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_4.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [70]:
model_result = np.array(model_4)
sorted_index = np.argsort(model_4)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fourth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 5 is 39.25780986995697
ROI for model 2 is 23.604388947105406
ROI for model 1 is 16.684378342056274
ROI for model 6 is 7.821926620292663
ROI for model 3 is 6.929837185287475
ROI for model 0 is 6.498544428062439
ROI for model 7 is -4.291380733489991
ROI for model 9 is -5.471038288307191
ROI for model 4 is -12.83416499595642
ROI for model 8 is -19.477455226898194


## Run 5

In [71]:
model_5 =[]

for i in range(len(fifth_10)):
    stocks = eval(fifth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_5.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [72]:
model_result = np.array(model_5)
sorted_index = np.argsort(model_5)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fifth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 19.17274286003113
ROI for model 6 is 14.763754214394092
ROI for model 2 is 14.460300617980955
ROI for model 9 is 9.323131063461304
ROI for model 5 is 7.828395192718506
ROI for model 3 is 6.353966648101807
ROI for model 7 is 0.10928088073730469
ROI for model 8 is -2.3114488819122316
ROI for model 4 is -4.287289756011963
ROI for model 0 is -10.670392717647552


## Run 6

In [73]:
model_6 =[]

for i in range(len(sixth_10)):
    stocks = eval(sixth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_6.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [74]:
model_result = np.array(model_6)
sorted_index = np.argsort(model_6)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'sixth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 2 is 35.07905251731873
ROI for model 0 is 14.99840526866913
ROI for model 3 is 12.04889541091919
ROI for model 7 is 10.618069512176513
ROI for model 4 is 10.290595821475982
ROI for model 1 is 4.389156093978882
ROI for model 9 is 3.693898580932617
ROI for model 8 is -5.004938960266114
ROI for model 5 is -5.390522392559052
ROI for model 6 is -6.139702890777588


## Run 7

In [75]:
model_7 =[]

for i in range(len(seventh_10)):
    stocks = eval(seventh_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_7.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [76]:
model_result = np.array(model_7)
sorted_index = np.argsort(model_7)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'seventh_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 3 is 18.91663444004059
ROI for model 1 is 18.603793176651003
ROI for model 0 is 18.151066918182373
ROI for model 2 is 16.506308568763732
ROI for model 5 is 13.400561906754971
ROI for model 6 is 11.079139847183228
ROI for model 4 is 7.24358906326294
ROI for model 9 is 6.791364957046508
ROI for model 7 is 3.5292602390289307
ROI for model 8 is 3.500861982727051


## Run 8

In [77]:
model_8 =[]

for i in range(len(eighth_10)):
    stocks = eval(eighth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_8.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [78]:
model_result = np.array(model_8)
sorted_index = np.argsort(model_8)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eighth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 2 is 25.188320442390445
ROI for model 0 is 12.241341979217529
ROI for model 7 is 11.762457683944701
ROI for model 4 is 9.051161225509643
ROI for model 6 is 8.230161577796936
ROI for model 3 is 7.242177558708191
ROI for model 9 is 7.055699501037598
ROI for model 5 is 5.989730619621277
ROI for model 1 is -1.463750687789917
ROI for model 8 is -25.303409716415402


## Run 9

In [79]:
model_9 =[]

for i in range(len(nineth_10)):
    stocks = eval(nineth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_9.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [80]:
model_result = np.array(model_9)
sorted_index = np.argsort(model_9)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'nineth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 4 is 32.81405608825684
ROI for model 6 is 21.171153318214415
ROI for model 9 is 20.667409469985962
ROI for model 1 is 19.25701202583313
ROI for model 2 is 8.066812220954896
ROI for model 5 is 5.647463815689087
ROI for model 8 is 4.5309082447052
ROI for model 7 is 2.4402492095947266
ROI for model 3 is 0.12479418601989746
ROI for model 0 is -4.595132967376709


## Run 10

In [81]:
model_10 =[]

for i in range(len(tenth_10)):
    stocks = eval(tenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_10.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [82]:
model_result = np.array(model_10)
sorted_index = np.argsort(model_10)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'tenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 22.641103551483155
ROI for model 4 is 20.840407652282714
ROI for model 7 is 15.122496290969847
ROI for model 6 is 11.316777758789062
ROI for model 0 is 11.182019899368287
ROI for model 3 is 9.49122915287018
ROI for model 8 is 3.752942258834839
ROI for model 2 is 0.5347695220947265
ROI for model 9 is 0.34903476276397705
ROI for model 5 is -4.963239894866944


## Run 11

In [83]:
model_11 =[]

for i in range(len(eleventh_10)):
    stocks = eval(eleventh_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_11.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [84]:
model_result = np.array(model_11)
sorted_index = np.argsort(model_11)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eleventh_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 6 is 19.673931837368013
ROI for model 0 is 12.11502876815796
ROI for model 3 is 9.890537396240234
ROI for model 5 is 7.464925067901611
ROI for model 7 is 7.436511565208435
ROI for model 2 is 2.049245839691162
ROI for model 9 is 1.1014653638839722
ROI for model 4 is 0.2337995288848877
ROI for model 1 is -2.492552811050415
ROI for model 8 is -8.29116314945221


## Run 12

In [85]:
model_12 =[]

for i in range(len(twelfth_10)):
    stocks = eval(twelfth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_12.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [86]:
model_result = np.array(model_12)
sorted_index = np.argsort(model_12)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'twelfth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 1 is 22.88420826511383
ROI for model 6 is 19.968026288604737
ROI for model 9 is 12.631148178958892
ROI for model 2 is 9.85612924156189
ROI for model 8 is 9.449837189102173
ROI for model 7 is 6.973273619461059
ROI for model 5 is -1.4132281463623046
ROI for model 0 is -6.231752734375
ROI for model 3 is -7.586270874595642
ROI for model 4 is -8.066174877357483


## Run 13

In [87]:
model_13 =[]

for i in range(len(thirteenth_10)):
    stocks = eval(thirteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_13.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [88]:
model_result = np.array(model_13)
sorted_index = np.argsort(model_13)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'thirteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 6 is 29.02257997932434
ROI for model 4 is 22.19297374010086
ROI for model 8 is 18.45455751457214
ROI for model 9 is 16.439353666687012
ROI for model 3 is 15.760495241546632
ROI for model 5 is 12.401059643173218
ROI for model 2 is 7.759382564163209
ROI for model 1 is -3.522229091262817
ROI for model 7 is -4.960001236724853
ROI for model 0 is -5.4537574495315555


## Run 14

In [89]:
model_14 =[]

for i in range(len(fourtheeth_10)):
    stocks = eval(fourtheeth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_14.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [90]:
model_result = np.array(model_14)
sorted_index = np.argsort(model_14)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fourtheeth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 7 is 32.51252196769715
ROI for model 5 is 24.544736160469053
ROI for model 4 is 16.876618280220033
ROI for model 2 is 9.398628291893004
ROI for model 9 is 9.272842681694032
ROI for model 1 is 5.45223532986641
ROI for model 3 is 0.5943329149246216
ROI for model 6 is 0.2289050453186035
ROI for model 8 is -1.6405910860061645
ROI for model 0 is -16.757186169052122


## Run 15

In [92]:
model_15 =[]

for i in range(len(fifteenth_10)):
    stocks = eval(fifteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_15.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [93]:
model_result = np.array(model_15)
sorted_index = np.argsort(model_15)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'fifteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 0 is 26.212934693717955
ROI for model 2 is 18.95993759765625
ROI for model 6 is 18.13652743110657
ROI for model 1 is 13.935343525123598
ROI for model 8 is 9.42920006351471
ROI for model 3 is 6.123630443954467
ROI for model 9 is 5.713704774856567
ROI for model 5 is 4.68878930683136
ROI for model 7 is 0.8931045671463013
ROI for model 4 is -17.775744325637817


## Run 16

In [95]:
model_16 =[]

for i in range(len(sixteenth_10)):
    stocks = eval(sixteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_16.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [96]:
model_result = np.array(model_16)
sorted_index = np.argsort(model_16)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'sixteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 9 is 32.433805068969725
ROI for model 0 is 19.90412138824463
ROI for model 1 is 17.54388905105591
ROI for model 7 is 17.33537530758381
ROI for model 3 is 9.840657750797272
ROI for model 5 is 9.464416567993164
ROI for model 8 is 8.298279631996154
ROI for model 2 is 8.120862814712524
ROI for model 6 is 6.6689121346473685
ROI for model 4 is -10.197560272598267


## Run 17

In [97]:
model_17 =[]

for i in range(len(seventeeth_10)):
    stocks = eval(seventeeth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_17.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [98]:
model_result = np.array(model_17)
sorted_index = np.argsort(model_17)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'seventeeth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 9 is 13.530442995071413
ROI for model 7 is 7.428481511116028
ROI for model 3 is 7.201279450416564
ROI for model 1 is 5.033760921287537
ROI for model 8 is 0.9694186988830567
ROI for model 5 is 0.8683442897796632
ROI for model 6 is -1.0678504558563233
ROI for model 2 is -5.10008341217041
ROI for model 0 is -10.754809588623047
ROI for model 4 is -11.005689860153197


## Run 18

In [100]:
model_18 =[]

for i in range(len(eighteenth_10)):
    stocks = eval(eighteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_18.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [101]:
model_result = np.array(model_18)
sorted_index = np.argsort(model_18)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'eighteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 4 is 14.632909018325806
ROI for model 3 is 9.455091925239563
ROI for model 5 is 8.274152027511597
ROI for model 9 is 8.09033117904663
ROI for model 6 is 5.799991591262818
ROI for model 8 is 5.220676239013672
ROI for model 2 is 0.3759200023651123
ROI for model 7 is -1.191715863609314
ROI for model 0 is -6.814510709953309
ROI for model 1 is -11.461577256393433


## Run 19

In [102]:
model_19 =[]

for i in range(len(nineteenth_10)):
    stocks = eval(nineteenth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_19.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [103]:
model_result = np.array(model_19)
sorted_index = np.argsort(model_19)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'nineteenth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 2 is 25.28007991390228
ROI for model 3 is 23.638054141807558
ROI for model 5 is 15.27008995399475
ROI for model 8 is 13.210675270462035
ROI for model 1 is 12.381200203132629
ROI for model 6 is 11.06266052722931
ROI for model 4 is 10.939964236450194
ROI for model 0 is 7.708525846099853
ROI for model 7 is -0.654261439704895
ROI for model 9 is -4.472897287845612


## 20th Run

In [104]:
model_20 =[]

for i in range(len(twentieth_10)):
    stocks = eval(twentieth_10.iloc[i])
    A = MA_RSI_Strategy(stocks, '2016-01-01', '2019-12-31',"1d", 500000, 0.00, False)
    A.run_strategy(50,200, 15, 70, 30, 0.25, 0.05)
    A.performance()
    A.performance_plots()
    model_20.append((A.capital - A.initial_capital) /A.initial_capital *100)

Output hidden; open in https://colab.research.google.com to view.

In [105]:
model_result = np.array(model_20)
sorted_index = np.argsort(model_20)
sorted_index = np.flip(sorted_index)
sorted_index = sorted_index.tolist()

top_20_models.append({'twentieth_10':{sorted_index[0],model_result[sorted_index[0]]}})
for index in sorted_index:
  print('ROI for model',str(index),"is", model_result[index])

ROI for model 3 is 12.96828636264801
ROI for model 2 is 12.185265453052521
ROI for model 0 is 10.59594848947525
ROI for model 7 is 3.7626916328430173
ROI for model 6 is 3.3828714916229248
ROI for model 4 is 2.348388720321655
ROI for model 9 is 1.3420229135513306
ROI for model 1 is -1.087897002029419
ROI for model 5 is -1.4047694593429565
ROI for model 8 is -7.326488187408447


In [114]:
top_20_models

[{'fourth_10': {5, 39.25780986995697}},
 {'fifth_10': {1, 19.17274286003113}},
 {'sixth_10': {2, 35.07905251731873}},
 {'seventh_10': {3, 18.91663444004059}},
 {'eighth_10': {2, 25.188320442390445}},
 {'nineth_10': {4, 32.81405608825684}},
 {'tenth_10': {1, 22.641103551483155}},
 {'eleventh_10': {6, 19.673931837368013}},
 {'twelfth_10': {1, 22.88420826511383}},
 {'thirteenth_10': {6, 29.02257997932434}},
 {'fourtheeth_10': {7, 32.51252196769715}},
 {'fifteenth_10': {0, 26.212934693717955}},
 {'sixteenth_10': {9, 32.433805068969725}},
 {'seventeeth_10': {9, 13.530442995071413}},
 {'eighteenth_10': {4, 14.632909018325806}},
 {'nineteenth_10': {2, 25.28007991390228}},
 {'twentieth_10': {3, 12.96828636264801}},
 {'first_10': {6, 30.592934116935734}},
 {'second_10': {9, 39.66862585971355}},
 {'third_10': {4, 19.200533485031126}}]